In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
from experiments.all import train_and_evaluate
from tsm.configuration import TSMConfigParser
import docker
import logging

logging.basicConfig(level=logging.INFO)

# load configuration
config = TSMConfigParser()
config.read("/home/pablo/tesis/tsm-pipeline/code/configs/pablo-bestia.cfg")
config.check()
docker_client = docker.from_env()

In [21]:
from scripts.docker import read_dbs_dataset
from numpy.random import RandomState
from random import sample

nosql_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/nosql_1.txt")
# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

sample_100 = rand.choice(nosql_dbs, size=100, replace=False)
sample_20 = rand.choice(nosql_dbs, size=20, replace=False)

In [22]:
outdir = "/home/pablo/dev-results/nosql-perf-experiment-100-run1"

results = train_and_evaluate(
    config=config,
    results_dir=outdir,
    query_type="nosql",
    train=list(sample_100),
    test=list(set(sample_20) - set(sample_100)),
    docker_client=docker_client,
)
results

INFO:experiments.all:train_and_evaluate with train size 100, test size 11
INFO:experiments.all:Running training
INFO:scripts.docker:creating /home/pablo/dev-results/nosql-perf-experiment-100-run1 dir. Omitting error if already existing
INFO:scripts.docker:creating /tmp/log dir. Omitting error if already existing
INFO:scripts.docker:running at container 75d12eb47261bef328f31587e134ed2173292786e213dc3617ebff8be28bc5e8. Use `docker logs 75d12eb47261bef328f31587e134ed2173292786e213dc3617ebff8be28bc5e8 --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /home/pablo/dev-results/nosql-perf-experiment-100-run1/*/reprScores.txt
INFO:misc.combinescores:working on 59 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /home/pablo/dev-results/nosql-perf-experiment-100-run1/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scrip

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.064516,0.888889,0.064,18,16,232,5,3.6


## mismos experimentos de perf con xss

In [24]:
xss_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/xss_1.txt")
len(xss_dbs)

90

In [25]:
xss_sample_100 = rand.choice(xss_dbs, size=70, replace=False)
xss_sample_20 = list(set(xss_dbs) - set(xss_sample_100))

In [28]:
outdir = "/home/pablo/dev-results/xss-perf-experiment-100-run1"

results = train_and_evaluate(
    config=config,
    results_dir=outdir,
    query_type="xss",
    train=list(xss_sample_100),
    test=xss_sample_20,
    docker_client=docker_client,
)
results

INFO:experiments.all:train_and_evaluate with train size 70, test size 20
INFO:experiments.all:Running training
INFO:scripts.docker:creating /home/pablo/dev-results/xss-perf-experiment-100-run1 dir. Omitting error if already existing
INFO:scripts.docker:creating /tmp/log dir. Omitting error if already existing
INFO:scripts.docker:running at container c94ffc3dd29525458529f766f1a4d39d610a414e9c09c552f3814ceb50279e2b. Use `docker logs c94ffc3dd29525458529f766f1a4d39d610a414e9c09c552f3814ceb50279e2b --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /home/pablo/dev-results/xss-perf-experiment-100-run1/*/reprScores.txt
INFO:misc.combinescores:working on 42 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /home/pablo/dev-results/xss-perf-experiment-100-run1/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.eval

CLIError: Failed to run query: Compiling query plan for /home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/XssEvaluation.ql.
[1/1] Found in cache: /home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/XssEvaluation.ql.
Starting evaluation of thepalbi/tsm-evaluation/src/XssEvaluation.ql.
Oops! A fatal internal error occurred. Details:
com.semmle.util.exception.CatastrophicError: An error occurred during the evaluation of GlobalAccessPaths#c651dae5::AccessPath::DominatingPaths::hasDominatingWrite#1#f/1@d1617apf
Could not load page 0 of BasicBlocks#7fe02f60::Internal::reachableBB#1#f/1@615a8bf4 (e8a50f97h4ja9jubi1h1jtb7tidj0)
The RA to evaluate was:

    {3} r1 = JOIN _DataFlow#d6e964f9::DataFlow::Node::asExpr#0#dispred#ff_10#join_rhs_GlobalAccessPaths#c651dae5::Acce__#shared WITH DataFlow#d6e964f9::DataFlow::PropRead#class#f ON FIRST 1 OUTPUT Lhs.1, Lhs.2, Lhs.0
    {2} r2 = JOIN r1 WITH GlobalAccessPaths#c651dae5::AccessPath::DominatingPaths::getAWriteBlock#2#ff